# Neuronale Netzwerke - Tutorial Teil 3
---

## 0. WIEDERHOLUNG: Typen neuronaler Netzwerke

### 0.1 Fully-Connected Neural Networks (FCN)

Ein __FCN__ (oder neuronale Netzwerke im Allgemeinen) kann als Sequenz spezifischer Schichten beschrieben werden. Diese Schichten, entsprechend der Position als _input_-, _output_- und _hidden layer_ bezeichnet, sind Kollektionen von Neuronen (siehe Bild). 
<img src="Bilder/fnc.jpeg">
Jedes Neuron hat bei einem __FCN__ eine Verbindung zu allen vorhergehenden und nachfolgenden Neuronen. Alle eingehenden (und die entsprechend ausgehenden) Verknüpfungen können beschrieben werden wie in folgendem Bild:
<img src="Bilder/single_neuron.jpg">
Jede Verknüpfung hat ein kennzeichnendes Gewicht, welches dem entsprechenden Eingangssignal eine Wichtigkeit zuordnet. Diese Miniprodukte werden dann summiert und durch eine Aktivierungsfunktion $f$ geschickt.
Somit führt jedes Neuron eine mathematische Operation der Form

\begin{equation}
y = f\left( b + \sum_{i=1}^n w_ix_i \right)
\end{equation}

durch.

---

### 0.2 Convolutional Neural Networks (CNN)

Ein __CNN__ ist eine Sammlung verschiedener _Kernels_ oder _Filter_ - kleine "Pakete" mit Gewichten, welche über das Bild geschoben (_gefaltet_) werden. Ziel ist es, das jedes einzelne Paket Pixelgruppen identifiziert, z.B. kleine Ecken und Linien in der ersten Schicht und komplizierte, kombinierte Strukturen in den Folgeschichten. Dies lässt sich gut an einem generellen Aufbaubeispiel verdeutlichen, z.B. hier:
<img src="Bilder/cnn.png">

oder hier:

<img src="Bilder/cnn2.png">

---

Die _Kernel_ werden dabei entsprechenden folgendem Bild über das Bild geschoben um sog. __feature maps__ zu generieren; funktionale Bilder welche die Auftrittswahrscheinlichkeit eines vom entsprechenden _Kernel_ codierten _features_ (Eigenschaft) am entsprechenden Ort angeben:

<img src="Bilder/conv_pic_example.png">

Häufig auftretend in solchen Strukturen sind __Pooling Layer__ , welche die vorhergehenden _feature maps_ zusammenfassen um Platz zu speichern und das Training zu beschleunigen. Ein solches _Layer_ ist definiert durch eine _Kernelgröße_, welche wie bei normalen _CNN_-Schichten über das Bild geschoben werden und dabei entsprechende Zusammenfassungsoperation durchführen, z.B. das Maximum oder das arithmetische Mittel.

---

### 0.3 Umsetzung in PyTorch

Um ein solches Netzwerk effektiv und vor allem einfach implementieren zu können, verwenden wir die __PyTorch__-Bibliothek (Installationsinformationen siehe vorheriges Tutorial).

Für ein _FCN_ sieht das beispielsweise so aus:

In [1]:
import torch.nn as nn
import torch
#torch.nn ist die Sammlung aller generellen Funktionen 
#eines neuronalen Netzwerkes, daher auch die Abküruzung nn.

class FCN_Base(nn.Module):
    def __init__(self):
        super(FCN_Base,self).__init__()
        self.fcn1 = nn.Linear(784,30)
        self.fcn2 = nn.Linear(30,30)
        self.fcn3 = nn.Linear(30,10)
        self.akt_func = nn.Softmax(dim=1)
        
    def forward(self,x):
        x = F.relu(self.fcn1(x))
        x = F.relu(self.fcn2(x))
        x = self.akt_func(self.fcn3(x))
        return x
    
#ODER

FCN_Base_2 = torch.nn.Sequential(
             torch.nn.Linear(784, 30), # Erste/Zweite Schicht: 784 Eingangsneuronen zu 30 "versteckten" Neuronen
             torch.nn.ReLU(),          # Aktivierungsfunktion
             torch.nn.Linear(30, 30),  # Zweite/Erste Schicht: 30 "versteckte" Neuronen gehen über zu 10 Ausgangsneuronen.
             torch.nn.ReLU(),
             torch.nn.Linear(30,10),
             torch.nn.Softmax(dim=1)   # Ausgangsaktivierungsfunktion. Siehe vorheriges Tutorial.
             )

Und für ein _CNN_:

In [2]:
#Sowie für CNNs:

class CNN_Base(nn.Module):
    def __init__(self):
        super(CNN_Base,self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3)  # 26x26
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3) # 24x24
        self.conv3 = nn.Conv2d(20, 30, kernel_size=3,padding=1)  # 12x12
        self.conv4 = nn.Conv2d(30, 30, kernel_size=3,padding=1)  # 12x12
        self.conv5 = nn.Conv2d(30, 50, kernel_size=3,padding=1) # 6x6        
        self.conv6 = nn.Conv2d(50, 50, kernel_size=3,padding=1)# 6x6                
        self.fc1 = nn.Linear(1800, 150)
        self.fc2 = nn.Linear(150, 50)
        self.fc3 = nn.Linear(50, 10)
        self.out_act = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = F.leaky_relu(F.max_pool2d(self.conv2(self.conv1(x)), 2))
        x = F.leaky_relu(F.max_pool2d(self.conv4(self.conv3(x)), 2))
        x = F.leaky_relu(self.conv6(self.conv5(x)), 2)
        x = x.view(-1, 1800)   #Hier ändern wir Bild- zu Vektorformat
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))        
        x = self.fc3(x)
        return self.out_act(x)

#ODER

class CNN_Base_2(nn.Module):
    def __init__(self):
        self.conv_teil = nn.Sequential(
                         nn.Conv2d(1,10,3),
                         nn.Conv2d(10,20,3),
                         nn.MaxPool2d(2),
                         nn.ReLU(),
                         nn.Conv2d(20,30,3),
                         nn.Conv2d(30,30,3),
                         nn.MaxPool2d(2),
                         nn.ReLU(),
                         nn.Conv2d(30,50,3),
                         nn.Conv2d(50,50,3),
                         nn.ReLU())
        self.fcn_teil =  nn.Sequential(
                         nn.Linear(1800,150),
                         nn.ReLU(),
                         nn.Linear(150,50),
                         nn.ReLU(),
                         nn.Linear(50,10),
                         nn.Softmax(dim=1))
    def forward(self,x):
        x = self.conv_teil(x)
        x = x.view(-1,1800)
        x = self.fcn_teil(x)
        return x
    

Für das allgemeine Training können wir das entsprechende Skript des letzten Tutorials verwenden:

#### 0.3.1 Generelle Funktionen:

In [4]:
import os, numpy as np
from torch.utils.data import Dataset

class MNIST_Data_Provider(Dataset):
    def __init__(self, all_image_paths, all_image_labels):
        super(MNIST_Data_Provider, self).__init__()

        self.all_image_paths, self.all_image_labels = all_image_paths, all_image_labels
        self.transform_to_torch_tensor = transforms.ToTensor()
        self.hot_list = np.eye(10).astype(int)     
        
    def one_hot(self, label):
        return self.hot_list[label]
    
    def __getitem__(self, idx):
        loaded_image    = Image.open(self.all_image_paths[idx])
        label_for_image = self.all_image_labels[idx]
        
        return self.transform_to_torch_tensor(loaded_image), label_for_image
    
    def __len__(self):
        return len(self.all_image_paths)
    

def get_image_paths(path_to_folder):
    all_image_paths = []
    all_labels      = []
    for numberpath in os.listdir(path_to_folder):
        if numberpath != ".DS_Store" and '_' not in numberpath:
            avail_img_paths = [x for x in os.listdir(path_to_folder+"/"+numberpath) if '._' not in x]
            all_image_paths.extend([path_to_folder+"/"+numberpath+"/"+x for x in avail_img_paths])
            all_labels.extend([int(numberpath) for _ in range(len(avail_img_paths))])
    return all_image_paths, all_labels  

#### 0.3.2 Trainings-Setup:

In [5]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torchvision import transforms
from PIL import Image
import os
import sys


""" Hyperparameter """
batch_size = 64
n_epochs   = 2
learning_rate = 0.0003
train_validation_split = 0.8


""" Aufsetzen der Datengeneratoren """
path_to_MNIST_dataset = os.getcwd()+"/MNIST/trainingSet"

all_image_paths, all_image_labels = get_image_paths(path_to_MNIST_dataset)

np.random.seed(1)
np.random.shuffle(all_image_paths)
np.random.seed(1)
np.random.shuffle(all_image_labels)

split_idx = int(len(all_image_paths)*train_validation_split)

training_img_paths = all_image_paths[:split_idx]
training_labels    = all_image_labels[:split_idx]
train_dataset = MNIST_Data_Provider(training_img_paths, training_labels)
train_datagen = DataLoader(train_dataset, batch_size=batch_size,drop_last=True, shuffle=True, num_workers=1)

validation_img_paths = all_image_paths[split_idx:]
validation_labels    = all_image_labels[split_idx:]
val_dataset = MNIST_Data_Provider(validation_img_paths, validation_labels)
val_datagen = DataLoader(val_dataset, batch_size=batch_size, drop_last=True, shuffle=False, num_workers=1)



""" Laden des Netzes, Aufsetzen des Optimierers"""
Net = CNN_Base()
device = torch.device('cpu')
# ### Im Falle ein GPU:
# device = torch.device('cuda')
_ = Net.to(device)
#Oder: FCN_Base, FCN_Base_2, CNN_Base_2


optimizer = optim.Adam(Net.parameters(), lr=learning_rate)

In [11]:
""" Training & Validierung """
for epoch in range(n_epochs):
    print("Training in Epoch {}...".format(epoch))
    
    """ Hier startet das Training! """
    Net.train()
    
    train_avg_loss = 0
    train_avg_acc  = 0
    
    data_coll = {"t_acc":[], "v_acc":[]}
    for idx, (img,label) in enumerate(train_datagen):
        img, label = img.to(device), label.to(device)
            
        optimizer.zero_grad()
        ### Für FCNs
        #output = Net(img.view(batch_size,-1))
        output = Net(img)
        
        loss = F.cross_entropy(output, label)
        
        loss.backward()
        optimizer.step()
        
        correct_guesses = output.cpu().detach().max(1)[1].eq(label.cpu().detach()).sum()
        
        train_avg_loss += loss.item()
        train_avg_acc  += correct_guesses.numpy()
        
        if idx%100==0 and idx!=0:
            print("\t T-Progress: [{}/{}]".format(idx+1,len(train_datagen)))

    train_avg_loss = train_avg_loss*1./(batch_size*len(train_datagen))
    train_avg_acc  = train_avg_acc*1./(batch_size*len(train_datagen))
    
    data_coll["t_acc"].append(train_avg_acc)
    
    
    """ Hier startet die Validierung """
    Net.eval()
    
    with torch.no_grad():
        val_avg_acc = 0
        for idx, (img,label) in enumerate(val_datagen):
            img, label = Variable(img), Variable(label)

            #output = Net(img.view(batch_size,-1))
            output = Net(img)
            correct_guesses = output.data.max(1)[1].eq(label.data).sum()

            val_avg_acc  += correct_guesses

            if idx%100==0 and idx!=0:
                print("\t V-Progress: [{}/{}]".format(idx+1,len(val_datagen)))    

        val_avg_acc = val_avg_acc*1./(batch_size*len(val_datagen))
        data_coll["v_acc"].append(val_avg_acc)    
        print("Results: T-Loss {0:2.5f} | T-Acc {1:3.4f}% | V-Acc {2:3.4f}%".format(train_avg_loss, train_avg_acc*100., val_avg_acc*100.))

Training in Epoch 0...
	 T-Progress: [101/525]
	 T-Progress: [201/525]
	 T-Progress: [301/525]
	 T-Progress: [401/525]
	 T-Progress: [501/525]
	 V-Progress: [101/131]
Results: T-Loss 0.02455 | T-Acc 88.9702% | V-Acc 0.0000%
Training in Epoch 1...
	 T-Progress: [101/525]
	 T-Progress: [201/525]
	 T-Progress: [301/525]
	 T-Progress: [401/525]
	 T-Progress: [501/525]
	 V-Progress: [101/131]
Results: T-Loss 0.02375 | T-Acc 94.0893% | V-Acc 0.0000%


---

### 0.4 Netzwerke speichern
Hier noch eine wichtige Erweiterung: Wenn die Gewichte des Netzwerk gespeichert werden sollen, um sie irgendwann nochmal wieder zu verwenden, bedient man sich einfach:

In [44]:
#torch.save(Net.state_dict(), _pfad_zum_speicherplatz_mit_namen)

Um ein Netzwerk mit den entsprechenden Gewichten auszustatten, verwendet man:

In [45]:
#Net.load_state_dict(torch.load(_pfad_zum_speicherplatz_mit_namen))

__Wichtig__: Die Netzwerkstruktur muss die Gleiche sein, um sie mit Gewichten ausstatten zu können.

Wenn wir dieses Netzwerk im Ordner `Netzwerke` speichern wollen, verwenden wir:

In [9]:
import os
### Falls der Ordner Netzwerke nicht existiert, erstellen wir ihn:
if not os.path.exists(os.getcwd()+'/Netzwerke'):
    os.makedirs(os.getcwd()+'/Netzwerke')
speichername = 'CNN_Version1'
torch.save(Net.state_dict(), os.getcwd()+'/Netzwerke/'+speichername)

Und um unser Netzwerke wieder mit den passenden Gewichten auszustatten, verwendet man

In [10]:
Net.load_state_dict(torch.load(os.getcwd()+'/Netzwerke/'+speichername))

---

## 1. Einige neue extra Methoden für bessere Performance

Unter <http://pytorch.org/docs/master/nn.html#> finden sich viele weitere Möglichkeiten, die Performance des Netzwerkes zu verbessern. Hier werden einige grundlegende Optionen aufgelistet.

### 1.1 Parameter Normen

Aufgrund der enormen Zahl an Parametern haben neuronale Netzwerke die Kapazität, sich den Trainingsdatensatz zu "merken". Das sorgt zwar für eine hohe Trainingsgenauigkeit, führt aber dazu das Vorhersagen außerhalb des Trainingssets unzuverlässig werden.

Um dem entgegenzuwirken, bedient man sich verschiedener Methoden, welche alle unter den Schirm __Regularisierung__ fallen.

Eine hiervon ist die Einführung von Parameternormen, welche Einschränkungen auf Netzwerkgewichte setzen, indem diese in die Kostenfunktion einbezogen werden:

* `L2`-__Norm__: $L_{reg}(\theta, \textbf{x}, \textbf{y}) = L(\theta, \textbf{x}, \textbf{y}) + \lambda \cdot ||\theta||^2_2 $
* `L1`-__Norm__: $L_{reg}(\theta, \textbf{x}, \textbf{y}) = L(\theta, \textbf{x}, \textbf{y}) + \lambda \cdot ||\theta||_2 $

Um die Kostenfunktion zu minimieren, muss das Netzwerk lernen, mit hohen Gewichten sparsam umzugehen. Intuitiv bedeutet das, dass das Netzwerk auf einzelne Elemente keine extreme Gewichtung setzen kann. Methematisch lässt sich für $L2$-Normen, zumindest für einfache Netzwerke, zeigen dass Gewichtungen entlang Richtungen niedriger Krümmung auf der Kostenlandschaft gedrückt werden. 

Im Falle der $L1$-Normen können diese sogar auf $0$ gelegt werden.

Der Hyperparameter gibt an, in welchem Maße man diese Zusatzkosten miteinbezogen werden sollen. Standardwerte hierfür liegen in den Bereichen $10^{-5}$.

Die Implementierung in PyTorch erfolgt direkt über den Optimierungsalgorithmuses (`L2` entspricht `weight_decay`):

In [40]:
#optimizer = optim.Adam(Net.parameters(), lr=learning_rate, weight_decay=1e-5)

### 1.2. Dropout

Dropout beruht auf der `Bagging`-Idee, in welcher ein Algorithmus auf verschiedenen Stichproben der Trainingsdaten generiert werden.
Die Implementation erfolgt, indem Neuronen für eine Trainingsiteration mit einer Wahrscheinlichkeit $p$ ausgeschaltet werden und den entsprechenden Input in dieser Iteration nicht "sehen". Zudem muss das Netzwerk lernen, mit den verbleibenden Gewichten sinnvolle Aussagen zu treffen, was der Entwicklung einzelner extremer Gewichtungen entgegenwirkt.

Während der eigentlichen Vorhersagen muss man allerdings alle Gewichte mit dem Faktor $\frac{1}{1-p}$ skalieren 
(PyTorch macht das automatisch), da bei $p$ ausgeschaltenen Neuronen während dem Training sich der Erwartungswert verschiebt wenn alle auf einmal verwendet werden.

PyTorch Version:

In [11]:
#dropout_layer = nn.Dropout(0.2)
#Angewandt in einem Netzwerk mit Input x:

# n1   = nn.Conv2d(in_channels, out_channels, kernel_size)
# n2   = nn.Conv2d(out_channels, out_channels, kernel_size)
# drop = nn.Dropout2d(0.2)
# x = n2(drop(n1(x)))

### 1.3. Optimierungsalgorithmen

Wie im vorherigen Tutorial erklärt, lassen sich verschiedene Optimierungsvarianten verwenden, die adaptive oder nicht-adaptiver Natur sein können. Daher hier eine simple Auflistung:

In [ ]:
#optim.Adam(), optim.SGD(momentum=0.9), optim.Adadelta, optim.RMSprop, optim.Adamax

### 1.4. Batch-Normalisierung

Batch-Normalization ist eine vergleichsweise neue Inklusion in die Welt der Netzwerke. Sie baut darauf auf das in Netzwerken die Verwendung von Minibatches zur Approximation des Gradienten folgendes Problem mit sich zieht:

Von einem Neuron in einer Schicht zur anderen können Aktivierungswerte starke Änderungen erfahren, die aber nicht zwingend der Verteilung entsprechen, derer sich die Eingangswerte unterziehen. Um dem Problem entgegenzuwirken, führt Batch-Normalisierung pro Schicht und Neuron lernbare Varianz- und Mittelwert-parameter ein, welche Aktivierung auf ein ähnliches Niveau skalieren. 

Dabei werden diese Parameter über den Minibatch abgeschätzt und mit jeder Trainingsiteration upgedatet. Für eine verlässliche Abschätzung ist es daher wichtig, dass Batchsizes groß genug gewählt werden ($8-16$ und aufwärts).

In einem Netzwerk wird das wie folgt eingebaut:

In [12]:
#bn = nn.BatchNorm2d(filter_in_next_layer)

# n1   = nn.Conv2d(in_channels, out_channels, kernel_size)
# n2   = nn.Conv2d(out_channels, out_channels, kernel_size)
# bn   = nn.BatchNorm2d(out_channels)
# x = n2(bn(n1(x)))

### 1.5. Aktivierungsfunktionen

Ebenso angesprochen waren die vielen Optionen für Aktivierungsfunktionen, die hier lediglich der Vollständigkeitshalber erwähnt werden sollen. Neben `ReLU` fallen darunter auch `LeakyReLU` (kleine Steigung bei Werten unter Null), `ParametricReLU` mit lernbarer Steigung für Werte unter Null) und viele andere Optionen.

In [42]:
#nn.ReLU, F.relu(), F.leaky_relu(), ...

### 1.6. Hyperparameter-Optimmierung

Final soll nochmal angemerkt werden, dass die Lernfähigkeit des Netzwerkes auch stark von der Wahl der Trainingshyperparameter abhängt. Darunter fallen beispielsweise:

* Lernrate
* Batchsize
* Trainings-Validierungseinteilung
* Wahrscheinlichkeit mit welcher Bilder aus den entsprechenden Klassen dem Netzwerk präsentiert werden
* ...

---

## Aufgabe

Das Ziel des ganzen Tages ist es, mindestens 3/4 verschiedene Netzwerkstrukturen zu trainieren, Trainings- und Validierungskurven zu speichern und zu vergleichen.